# 🚀 VoiceFlow Intelligence — Comprehensive Blockchain Data Collection
### NYU Buildathon 2026 | Feb 20–22 | Team: Nikhil, Aman, Dhanush

---

## 📋 NOTEBOOK OVERVIEW

This notebook collects **~25 GB of encyclopedic blockchain/crypto data** to power the **VoiceFlow Intelligence** RAG pipeline and fine-tuning datasets.  
Run it **before the hackathon starts** and store the output folder on a USB drive.

### Why Pre-Collect?
- ⏱️ Zero time wasted on data scraping during the event
- 🏆 Encyclopedic domain knowledge impresses judges and wins sponsor prizes  
- 📡 API rate limits will throttle you live — avoid this entirely
- 🎯 Pre-built RAG index = 8–12 extra hours for coding & polish

### Prize Strategy
| Data Category | Sponsor Unlocked | Prize Value |
|---|---|---|
| Binance OHLCV + technical indicators | TrueMarkets + FinTech track | $500–$1 000 |
| CoinGecko 13 000-coin profiles | Databricks analytics showcase | $3 000 |
| DeFi Llama protocols + yield pools | Databricks ML + FinTech | $3 000 |
| OpenAssistant conversations | ElevenLabs voice dialogue | $7 920 |
| ArXiv whitepapers + Wikipedia | Technical depth / Best Overall | $2 000 |

### Sections
1. Environment setup  
2. Binance OHLCV price data (~5 GB)  
3. CoinGecko encyclopedia (~2.5 GB)  
4. DeFi ecosystem via DeFi Llama (~2.2 GB)  
5. Protocol whitepapers — ArXiv + Wikipedia (~5.5 GB)  
6. Regulatory & TrueMarkets docs (~1.1 GB)  
7. Crypto news archives (~4 GB)  
8. Conversation training data — OpenAssistant (~1.5 GB)  
9. NFT & Web3 data (~3 GB)  
10. TrueMarkets live API client  
11. Build ChromaDB RAG index  
12. Validation report & cheat sheet  

⚠️ **Estimated total time:** 8–10 h | **Estimated size:** ~25 GB

---
## Section 1 — Environment Setup

Install every library upfront so nothing breaks at 2 AM during the hackathon.

In [ ]:
# ─── CELL 1.1 — Install Dependencies ────────────────────────────────────────
# One-shot install of every library used across all sections.

!pip install -q \
    requests pandas numpy tqdm \
    python-binance pycoingecko \
    arxiv wikipedia-api \
    datasets beautifulsoup4 lxml \
    gitpython \
    chromadb sentence-transformers \
    PyPDF2 aiohttp websockets nest_asyncio \
    python-dotenv tenacity \
    pyarrow fastparquet

print("✅ All dependencies installed")

In [ ]:
# ─── CELL 1.2 — Global Config & Directory Structure ─────────────────────────
import os, json, time, logging, hashlib
from datetime import datetime
from pathlib import Path

import requests
import pandas as pd
import numpy as np
from tqdm import tqdm

# ── Configure these before running ──────────────────────────────────────────
USB_MODE = False                         # Set True to write directly to USB
USB_PATH = "/Volumes/HACKATHON"          # macOS USB path
BASE_DIR = USB_PATH if USB_MODE else "./hackathon_data"

DIRS = {
    "prices":       f"{BASE_DIR}/01_market_prices",
    "coingecko":    f"{BASE_DIR}/02_coingecko_encyclopedia",
    "defi":         f"{BASE_DIR}/03_defi_ecosystem",
    "protocols":    f"{BASE_DIR}/04_protocol_whitepapers",
    "regulatory":   f"{BASE_DIR}/05_regulatory_docs",
    "news":         f"{BASE_DIR}/06_crypto_news",
    "conversation": f"{BASE_DIR}/07_conversation_training",
    "nft":          f"{BASE_DIR}/08_nft_web3",
    "truemarkets":  f"{BASE_DIR}/09_truemarkets_live",
    "rag_index":    f"{BASE_DIR}/10_rag_chromadb",
    "logs":         f"{BASE_DIR}/logs",
}

for p in DIRS.values():
    Path(p).mkdir(parents=True, exist_ok=True)

# ── Logging ──────────────────────────────────────────────────────────────────
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s [%(levelname)s] %(message)s",
    handlers=[
        logging.FileHandler(f"{DIRS['logs']}/collection.log"),
        logging.StreamHandler(),
    ],
)
log = logging.getLogger("voiceflow")

# ── API keys (fill in before running) ────────────────────────────────────────
# Binance:   https://www.binance.us/account/api  (optional — public endpoints work without)
# CoinGecko: https://www.coingecko.com/en/api    (optional — free tier is enough)
BINANCE_API_KEY    = os.getenv("BINANCE_API_KEY",    "")
BINANCE_API_SECRET = os.getenv("BINANCE_API_SECRET", "")
COINGECKO_API_KEY  = os.getenv("COINGECKO_API_KEY",  "")

# ── Manifest: track every file we collect ────────────────────────────────────
MANIFEST_PATH = f"{BASE_DIR}/manifest.json"
manifest = {"started_at": datetime.now().isoformat(), "sections": {}}

def save_manifest():
    manifest["updated_at"] = datetime.now().isoformat()
    with open(MANIFEST_PATH, "w") as f:
        json.dump(manifest, f, indent=2)

def file_size_mb(path):
    try:    return round(os.path.getsize(path) / 1024 / 1024, 2)
    except: return 0

def dir_size_mb(path):
    total = 0
    for dp, _, fnames in os.walk(path):
        for fn in fnames:
            total += os.path.getsize(os.path.join(dp, fn))
    return round(total / 1024 / 1024, 2)

log.info(f"Output directory: {BASE_DIR}")
print(f"✅ Config ready — writing to: {BASE_DIR}")

---
## Section 2 — Binance OHLCV Price Data

### Why this data?
| Use case | Detail |
|---|---|
| **Databricks prize** | Store as Delta Lake Parquet; track feature engineering in MLflow |
| **Nemotron** | Feed structured OHLCV + indicators — math-capable model handles RSI/MACD/BB calc |
| **TrueMarkets** | Historical context so voice agent says *"BTC is up 3.2% today, last at $87 420"* |
| **ElevenLabs** | Factually grounded price references make voice responses sound authoritative |

### What we collect
10 major pairs × 365 days × 1-minute candles ≈ **~5.25 million rows** saved as snappy-compressed Parquet files.  
Parquet is 5–10× smaller than CSV and loads instantly in Databricks Spark jobs.

In [ ]:
# ─── CELL 2.1 — Fetch Binance 1-Minute OHLCV ─────────────────────────────────
# Binance public klines endpoint — NO API key required.
# We page through 1 000-candle chunks until the full year is downloaded.
#
# Column meanings:
#   open/high/low/close → price in quote asset (USDT)
#   volume             → base asset volume
#   quote_volume       → notional traded (≈ USD value)
#   trades             → number of individual trades in that minute
#   taker_buy_base/quote → volume bought by the aggressive (taker) side
#
# WHY 1-MINUTE CANDLES?
#   Granular enough to calculate any higher-timeframe indicator.
#   A 1-hour candle = resample 60 rows. A daily candle = 1 440 rows.
#   Gives Databricks a real workload to showcase parallelism.

from datetime import timedelta

BINANCE_BASE = "https://api.binance.us/api/v3"

SYMBOLS   = ["BTCUSDT","ETHUSDT","SOLUSDT","BNBUSDT","XRPUSDT",
             "ADAUSDT","AVAXUSDT","MATICUSDT","LINKUSDT","UNIUSDT"]
INTERVAL  = "1m"
DAYS_BACK = 365
LIMIT     = 1000

def fetch_klines(symbol, interval, start_ms, end_ms):
    r = requests.get(
        f"{BINANCE_BASE}/klines",
        params={"symbol": symbol, "interval": interval,
                "startTime": start_ms, "endTime": end_ms, "limit": LIMIT},
        timeout=30,
    )
    r.raise_for_status()
    return r.json()

def collect_symbol_ohlcv(symbol):
    out = f"{DIRS['prices']}/{symbol}_{INTERVAL}_{DAYS_BACK}d.parquet"
    if os.path.exists(out):
        log.info(f"  Skipping {symbol} — already collected")
        return out

    end_ms   = int(datetime.utcnow().timestamp() * 1000)
    start_ms = int((datetime.utcnow() - timedelta(days=DAYS_BACK)).timestamp() * 1000)
    candles, cursor = [], start_ms

    with tqdm(total=DAYS_BACK * 1440, desc=f"  {symbol}", unit="candles") as pbar:
        while cursor < end_ms:
            try:
                batch = fetch_klines(symbol, INTERVAL, cursor, end_ms)
            except requests.RequestException as e:
                log.warning(f"  Rate limited ({symbol}): {e}. Retrying in 10 s...")
                time.sleep(10)
                continue
            if not batch:
                break
            candles.extend(batch)
            cursor = batch[-1][6] + 1   # close_time of last candle + 1 ms
            pbar.update(len(batch))
            time.sleep(0.12)            # ≈ 8 req/s — well under Binance 1 200/min limit

    cols = ["open_time","open","high","low","close","volume",
            "close_time","quote_volume","trades",
            "taker_buy_base","taker_buy_quote","_ignore"]
    df = pd.DataFrame(candles, columns=cols)
    for c in ["open","high","low","close","volume","quote_volume",
              "taker_buy_base","taker_buy_quote"]:
        df[c] = df[c].astype(float)
    df["trades"]   = df["trades"].astype(int)
    df["open_dt"]  = pd.to_datetime(df["open_time"],  unit="ms", utc=True)
    df["close_dt"] = pd.to_datetime(df["close_time"], unit="ms", utc=True)
    df["symbol"]   = symbol
    df.drop(columns=["_ignore"], inplace=True)
    df.to_parquet(out, index=False, compression="snappy")
    log.info(f"  ✅ {symbol}: {len(df):,} candles → {file_size_mb(out)} MB")
    return out

print("=" * 60)
print("SECTION 2: Binance OHLCV collection")
print("=" * 60)
price_paths = [collect_symbol_ohlcv(s) for s in SYMBOLS]
manifest["sections"]["prices"] = {"symbols": SYMBOLS,
                                   "total_mb": dir_size_mb(DIRS["prices"])}
save_manifest()
print(f"\n✅ Price data complete — {dir_size_mb(DIRS['prices'])} MB")

In [ ]:
# ─── CELL 2.2 — Technical Indicators ─────────────────────────────────────────
# Pre-computing indicators has two benefits:
#   1. Nemotron gets LABELLED features ("RSI(14)=68.4") not raw numbers.
#      This activates its multi-step mathematical reasoning capability.
#   2. Databricks demo: register a feature store in Unity Catalog and
#      track a feature engineering job in MLflow Experiments.
#
# INDICATOR REFERENCE:
#   RSI(14)       → 0–100 momentum oscillator; >70 = overbought, <30 = oversold
#   SMA(20)       → Simple 20-period baseline; acts as dynamic support/resistance
#   EMA(12/26)    → Exponential MAs that weight recent prices more heavily
#   MACD          → EMA(12)–EMA(26); crossover = momentum shift signal
#   MACD signal   → EMA(9) of MACD; generates buy/sell crossover signals
#   MACD hist     → MACD–signal; histogram shows divergence strength
#   BB upper/lower→ SMA(20) ± 2σ; price near upper = expensive, near lower = cheap
#   BB width      → Normalised band width; spikes = high volatility regimes
#   VWAP(60m)     → Volume-weighted average price over last hour;
#                   TrueMarkets uses VWAP for institutional order routing

def add_indicators(df):
    c, v = df["close"], df["volume"]

    delta = c.diff()
    gain  = delta.clip(lower=0).rolling(14).mean()
    loss  = (-delta.clip(upper=0)).rolling(14).mean()
    df["rsi_14"]      = 100 - 100 / (1 + gain / loss.replace(0, 1e-10))

    df["sma_20"]      = c.rolling(20).mean()
    df["ema_12"]      = c.ewm(span=12, adjust=False).mean()
    df["ema_26"]      = c.ewm(span=26, adjust=False).mean()
    df["macd"]        = df["ema_12"] - df["ema_26"]
    df["macd_signal"] = df["macd"].ewm(span=9, adjust=False).mean()
    df["macd_hist"]   = df["macd"] - df["macd_signal"]

    std = c.rolling(20).std()
    df["bb_upper"]    = df["sma_20"] + 2 * std
    df["bb_lower"]    = df["sma_20"] - 2 * std
    df["bb_width"]    = (df["bb_upper"] - df["bb_lower"]) / df["sma_20"]

    tp = (df["high"] + df["low"] + c) / 3
    df["vwap_60m"]    = (tp * v).rolling(60).sum() / v.rolling(60).sum()

    df["pct_1m"]  = c.pct_change(1)    * 100
    df["pct_1h"]  = c.pct_change(60)   * 100
    df["pct_1d"]  = c.pct_change(1440) * 100
    return df

for sym in SYMBOLS:
    src = f"{DIRS['prices']}/{sym}_{INTERVAL}_{DAYS_BACK}d.parquet"
    if not os.path.exists(src):
        print(f"  ⚠️  {sym} raw file missing — run Cell 2.1 first")
        continue
    df  = pd.read_parquet(src)
    df  = add_indicators(df)
    out = f"{DIRS['prices']}/{sym}_features.parquet"
    df.to_parquet(out, index=False, compression="snappy")
    print(f"  ✅ {sym} features → {file_size_mb(out)} MB")

print("\n✅ Technical indicators complete")

---
## Section 3 — CoinGecko Encyclopedia (13 000+ Coins)

### Why this data?
This is the **encyclopedic layer** — what separates a toy demo from a production-grade agent.

When a judge asks *"Tell me about Render Network"* (coin #847) or *"What consensus does Fantom use?"*, the agent needs this.

### What we collect per coin
- Full text description (technology, use case, team)
- Category tags (DeFi, NFT, L1, L2, Gaming, PoW, PoS…)
- Market data: cap, volume, ATH, ATL, supply
- Developer stats: GitHub commits, contributors, issues
- Social stats: Twitter followers, Reddit subscribers
- Exchange listings and trading pairs

In [ ]:
# ─── CELL 3.1 — CoinGecko: Full Market List ──────────────────────────────────
# Strategy: page through /coins/markets (250 per page) for a lightweight
# market-data snapshot of every listed coin, then call /coins/{id} for
# the richer profile data we need for RAG.
#
# Rate limits:
#   Free tier ≈ 10–30 calls/min  → sleep 6 s between calls
#   Pro tier  ≈ 500 calls/min   → sleep 0.12 s
# We sleep conservatively to avoid 429s during overnight collection.

from pycoingecko import CoinGeckoAPI

cg = CoinGeckoAPI(api_key=COINGECKO_API_KEY if COINGECKO_API_KEY else None)

def get_all_coins_market_data():
    out = f"{DIRS['coingecko']}/all_coins_market.parquet"
    if os.path.exists(out):
        log.info("  Skipping CoinGecko market list — already collected")
        return pd.read_parquet(out)

    all_coins, page = [], 1
    while True:
        try:
            batch = cg.get_coins_markets(
                vs_currency="usd", order="market_cap_desc",
                per_page=250, page=page, sparkline=False,
                price_change_percentage="24h,7d,30d",
            )
        except Exception as e:
            log.warning(f"  Page {page} error: {e}. Sleeping 30 s...")
            time.sleep(30)
            continue
        if not batch:
            break
        all_coins.extend(batch)
        print(f"  Page {page}: +{len(batch)} coins (total {len(all_coins)})")
        page += 1
        time.sleep(6)

    df = pd.DataFrame(all_coins)
    df.to_parquet(out, index=False, compression="snappy")
    print(f"\n  ✅ {len(df)} coins saved")
    return df

coins_df = get_all_coins_market_data()
print(f"Total coins collected: {len(coins_df):,}")
print(coins_df[["name","symbol","market_cap_rank","current_price"]].head(10))

In [ ]:
# ─── CELL 3.2 — CoinGecko: Rich Coin Profiles ────────────────────────────────
# For each coin we store the FULL profile which becomes a RAG document:
#
#   title:   "Bitcoin (BTC)"
#   text:    "Bitcoin is a decentralised digital currency...
#             Categories: Cryptocurrency, PoW, Store of Value
#             Market Cap: $1.7 T | Rank: #1 | ATH: $108 000
#             Algorithm: SHA-256 | Block time: 10 min"
#
# We use a done_ids.txt checkpoint file so the run is fully resumable
# if your connection drops or rate limit hits mid-collection.

def fetch_coin_profile(coin_id):
    try:
        d = cg.get_coin_by_id(
            coin_id, localization=False, tickers=False,
            market_data=True, community_data=True,
            developer_data=True, sparkline=False,
        )
        return {
            "id":            coin_id,
            "name":          d.get("name"),
            "symbol":        d.get("symbol"),
            "description":   (d.get("description") or {}).get("en", "")[:2000],
            "categories":    d.get("categories", []),
            "homepage":      (d.get("links") or {}).get("homepage", [None])[0],
            "github":        (d.get("links") or {}).get("repos_url", {}).get("github", []),
            "market_cap_usd":    (d.get("market_data") or {}).get("market_cap", {}).get("usd"),
            "current_price_usd": (d.get("market_data") or {}).get("current_price", {}).get("usd"),
            "ath_usd":           (d.get("market_data") or {}).get("ath", {}).get("usd"),
            "genesis_date":      d.get("genesis_date"),
            "hashing_algorithm": d.get("hashing_algorithm"),
            "block_time_minutes":d.get("block_time_in_minutes"),
            "total_supply":      (d.get("market_data") or {}).get("total_supply"),
            "circulating_supply":(d.get("market_data") or {}).get("circulating_supply"),
            "developer_score":   d.get("developer_score"),
            "community_score":   d.get("community_score"),
        }
    except Exception as e:
        log.warning(f"  Could not fetch {coin_id}: {e}")
        return None

def collect_coin_profiles(coins_df, max_coins=3000):
    out_path  = f"{DIRS['coingecko']}/coin_profiles.jsonl"
    done_path = f"{DIRS['coingecko']}/done_ids.txt"

    done_ids = set()
    if os.path.exists(done_path):
        done_ids = set(open(done_path).read().splitlines())
        print(f"  Resuming: {len(done_ids)} profiles cached")

    targets = coins_df.sort_values("market_cap_rank").head(max_coins)["id"].tolist()
    remaining = [cid for cid in targets if cid not in done_ids]
    print(f"  Fetching {len(remaining)} profiles...")

    with open(out_path, "a") as out_f, open(done_path, "a") as done_f:
        for cid in tqdm(remaining, desc="  Coin profiles"):
            profile = fetch_coin_profile(cid)
            if profile:
                out_f.write(json.dumps(profile) + "\n")
                done_f.write(cid + "\n")
            time.sleep(6)   # Free-tier rate limit

    total = len(done_ids) + len(remaining)
    print(f"  ✅ {total} profiles saved")

collect_coin_profiles(coins_df, max_coins=3000)
manifest["sections"]["coingecko"] = {"total_mb": dir_size_mb(DIRS["coingecko"])}
save_manifest()

---
## Section 4 — DeFi Ecosystem (DeFi Llama)

### Why this data?
DeFi is the most technically complex domain judges probe on.  
DeFi Llama has a **completely free, no-auth API** covering 3 000+ protocols.

Voice agent capabilities this unlocks:
- *"What's the TVL on Aave vs Compound?"* → real numbers
- *"Which chain has the most DeFi activity?"* → chain breakdown
- *"What's the best USDC yield right now?"* → live pool APYs

### What we collect
- All protocol metadata + TVL + category
- Historical TVL time series for top 200 (Databricks time-series model)
- Live yield pool APYs across every DeFi lending/liquidity protocol

In [ ]:
# ─── CELL 4.1 — DeFi Llama: Protocols + Yield Pools ─────────────────────────
# DeFi Llama protocol record includes:
#   name, slug, category (Dexes/Lending/CDP/Derivatives/Yield/Bridge/Stablecoins…)
#   tvl (USD), change_1h/1d/7d, chains (where it's deployed), description,
#   github, twitter, forkedFrom (which protocol it's a fork of)
#
# forkedFrom is GOLD for the agent:
#   "SushiSwap is a fork of Uniswap V2, adding SUSHI token rewards..."
#
# YIELD POOL DATA (from yields.llama.fi):
#   pool, project, chain, symbol, tvlUsd, apy, apyBase, apyReward,
#   underlyingTokens, rewardTokens, apyPct7D, apyPct30D
#   → agent can compare real-time yields across protocols

LLAMA_BASE = "https://api.llama.fi"

def collect_defi_protocols():
    out = f"{DIRS['defi']}/all_protocols.parquet"
    if os.path.exists(out):
        log.info("  Skipping DeFi protocols — already collected")
        return pd.read_parquet(out)

    resp = requests.get(f"{LLAMA_BASE}/protocols", timeout=30)
    resp.raise_for_status()
    data = resp.json()

    rows = [{
        "name":       p.get("name"),
        "slug":       p.get("slug"),
        "url":        p.get("url"),
        "description":str(p.get("description", ""))[:1000],
        "chain":      p.get("chain"),
        "chains":     json.dumps(p.get("chains", [])),
        "category":   p.get("category"),
        "tvl":        p.get("tvl", 0),
        "change_1h":  p.get("change_1h", 0),
        "change_1d":  p.get("change_1d", 0),
        "change_7d":  p.get("change_7d", 0),
        "github":     json.dumps(p.get("github", [])),
        "forkedFrom": json.dumps(p.get("forkedFrom", [])),
        "twitter":    p.get("twitter"),
        "mcap":       p.get("mcap"),
    } for p in data]

    df = pd.DataFrame(rows)
    df.to_parquet(out, index=False, compression="snappy")
    print(f"  ✅ {len(df)} protocols saved ({file_size_mb(out)} MB)")
    return df

def collect_yield_pools():
    out = f"{DIRS['defi']}/yield_pools.parquet"
    if os.path.exists(out):
        print("  Skipping yield pools — already collected"); return
    resp = requests.get("https://yields.llama.fi/pools", timeout=30)
    resp.raise_for_status()
    df = pd.DataFrame(resp.json().get("data", []))
    df.to_parquet(out, index=False, compression="snappy")
    print(f"  ✅ {len(df)} yield pools saved ({file_size_mb(out)} MB)")

def collect_tvl_history(slug):
    out = f"{DIRS['defi']}/tvl_history/{slug}.json"
    Path(f"{DIRS['defi']}/tvl_history").mkdir(exist_ok=True)
    if os.path.exists(out): return
    try:
        r = requests.get(f"{LLAMA_BASE}/protocol/{slug}", timeout=15)
        r.raise_for_status()
        d = r.json()
        with open(out, "w") as f:
            json.dump({"slug": slug, "name": d.get("name"),
                       "category": d.get("category"),
                       "tvl": d.get("tvl", []),
                       "chainTvls": d.get("chainTvls", {})}, f)
    except Exception as e:
        log.warning(f"  TVL history error for {slug}: {e}")

print("=" * 60)
print("SECTION 4: DeFi Ecosystem")
print("=" * 60)
protocols_df = collect_defi_protocols()
collect_yield_pools()

top200 = protocols_df.nlargest(200, "tvl")["slug"].dropna().tolist()
print(f"\nCollecting TVL history for top 200 protocols...")
for slug in tqdm(top200, desc="  TVL history"):
    collect_tvl_history(slug)
    time.sleep(0.5)

manifest["sections"]["defi"] = {"protocols": len(protocols_df),
                                 "total_mb": dir_size_mb(DIRS["defi"])}
save_manifest()
print(f"\n✅ DeFi data complete — {dir_size_mb(DIRS['defi'])} MB")

---
## Section 5 — Protocol Whitepapers & Technical Documentation

### Why this data?
Technical credibility is what separates 1st place from 2nd.  
Judges drill into specifics: *"How does Solana's Proof of History work?"* — if the agent stumbles, you lose.

### Sources
| Source | What it provides |
|---|---|
| **ArXiv** | Peer-reviewed papers on consensus, ZK proofs, AMMs, MEV, bridges |
| **Wikipedia** | Accessible explanations; great for ElevenLabs voice synthesis |

In [ ]:
# ─── CELL 5.1 — ArXiv Academic Papers ────────────────────────────────────────
# 17 targeted queries × 50 results each ≈ 850 papers.
# We store abstract + metadata — sufficient for RAG without downloading full PDFs.
#
# Query design: specific technical terms → high-precision results
# e.g. "optimistic rollup fraud proof layer 2 scaling" targets the exact
# concept rather than generic "ethereum scaling" (which returns too much noise)

import arxiv

ARXIV_QUERIES = [
    "blockchain consensus mechanism proof of stake",
    "Bitcoin proof of work nakamoto consensus",
    "Ethereum smart contract EVM execution",
    "optimistic rollup fraud proof layer 2 scaling",
    "zero knowledge proof zkSNARK zkSTARK blockchain",
    "state channel payment channel lightning network",
    "automated market maker AMM DEX constant product formula",
    "DeFi lending protocol liquidation overcollateralization",
    "maximal extractable value MEV flashbot blockchain",
    "stablecoin algorithmic collateralized mechanism",
    "elliptic curve cryptography digital signature blockchain",
    "hash function SHA256 merkle tree blockchain",
    "cross chain bridge interoperability protocol",
    "blockchain oracle price feed manipulation",
    "NFT non-fungible token smart contract ERC-721",
    "DAO governance voting mechanism token weighted",
    "tokenomics mechanism design crypto incentives",
]

def collect_arxiv_papers():
    out = f"{DIRS['protocols']}/arxiv_papers.jsonl"
    seen = set()
    if os.path.exists(out):
        with open(out) as f:
            seen = {json.loads(l)["id"] for l in f}
        print(f"  Resuming: {len(seen)} papers cached")

    new_count = 0
    with open(out, "a") as f:
        for q in tqdm(ARXIV_QUERIES, desc="  ArXiv queries"):
            search = arxiv.Search(query=q, max_results=50,
                                  sort_by=arxiv.SortCriterion.Relevance)
            for p in search.results():
                pid = p.entry_id.split("/")[-1]
                if pid in seen: continue
                f.write(json.dumps({
                    "id":        pid,
                    "title":     p.title,
                    "abstract":  p.summary,
                    "authors":   [str(a) for a in p.authors],
                    "published": p.published.isoformat() if p.published else None,
                    "categories":p.categories,
                    "url":       p.entry_id,
                    "query":     q,
                }) + "\n")
                seen.add(pid)
                new_count += 1
            time.sleep(3)   # ArXiv polite rate limit

    print(f"  ✅ {new_count} new papers saved (total {len(seen)})")
    return out

arxiv_path = collect_arxiv_papers()

In [ ]:
# ─── CELL 5.2 — Wikipedia Crypto Articles ────────────────────────────────────
# Wikipedia gives the agent:
#   1. Accessible explanations — ideal for ElevenLabs voice (natural sentences,
#      not bullet-point jargon from whitepapers)
#   2. Historical timelines — FTX collapse, 2017 ICO mania, Mt. Gox
#   3. Cross-concept links — agent can chain from "DeFi" → "AMM" → "Uniswap"
#
# We cap each article at 50 000 chars (≈ 12 500 tokens) — more than enough
# for RAG retrieval while keeping storage manageable.

import wikipediaapi

WIKI_TOPICS = [
    "Cryptocurrency","Blockchain","Bitcoin","Ethereum","Smart_contract",
    "Decentralized_finance","Non-fungible_token","Web3",
    "Decentralized_autonomous_organization",
    "Proof_of_work","Proof_of_stake","Delegated_proof_of_stake",
    "Byzantine_fault","Merkle_tree","Hash_function","SHA-2",
    "Elliptic-curve_cryptography","Zero-knowledge_proof",
    "Lightning_Network","Ethereum_2.0",
    "Solana_(blockchain)","Avalanche_(blockchain_platform)",
    "Cardano_(blockchain_platform)","Polkadot_(blockchain_platform)",
    "Cosmos_(blockchain)","Ripple_(payment_protocol)",
    "Binance_Smart_Chain","Polygon_(blockchain)",
    "Uniswap","Automated_market_maker","Yield_farming",
    "Liquidity_mining","Decentralized_exchange",
    "Stablecoin","Tether_(cryptocurrency)","USD_Coin","Dai_(cryptocurrency)",
    "Cryptocurrency_exchange","Initial_coin_offering","Crypto_winter",
    "Bitcoin_halving","Mt._Gox","FTX_(company)","Celsius_Network",
    "Cryptocurrency_and_crime","Bitcoin_in_El_Salvador","Cryptocurrency_bubble",
]

def collect_wikipedia():
    out = f"{DIRS['protocols']}/wikipedia_articles.jsonl"
    done = set()
    if os.path.exists(out):
        with open(out) as f:
            done = {json.loads(l)["title"] for l in f}
        print(f"  Resuming: {len(done)} articles cached")

    wiki = wikipediaapi.Wikipedia(
        user_agent="VoiceFlowIntelligence/1.0 (NYU Buildathon 2026)",
        language="en",
    )
    new_count = 0
    with open(out, "a") as f:
        for topic in tqdm(WIKI_TOPICS, desc="  Wikipedia"):
            if topic in done: continue
            page = wiki.page(topic)
            if not page.exists():
                log.warning(f"  Not found: {topic}"); continue
            f.write(json.dumps({
                "title":     page.title,
                "summary":   page.summary[:2000],
                "full_text": page.text[:50000],
                "url":       page.fullurl,
                "sections":  [s.title for s in page.sections],
            }) + "\n")
            new_count += 1
            time.sleep(1)

    print(f"  ✅ {new_count} new Wikipedia articles saved")
    return out

wiki_path = collect_wikipedia()
manifest["sections"]["protocols"] = {"total_mb": dir_size_mb(DIRS["protocols"])}
save_manifest()

---
## Section 6 — Regulatory & TrueMarkets Documentation

### Why this data?
TrueMarkets is a sponsor with a full Rule Book. The voice agent must answer:
- *"Why was my order rejected?"* → price band, min order value, tick size
- *"What's the self-trade prevention behaviour?"* → cancel-aggressive vs cancel-both
- *"Explain the DeFi Gateway order ingress rules"* → Appendix A

We chunk the docs at ~1 000 chars with 100-char overlap so RAG retrieval hits the right paragraph, not a 10-page wall of text.

In [ ]:
# ─── CELL 6.1 — TrueMarkets Documentation Ingestion ─────────────────────────
# The 14 TrueMarkets .txt files in /mnt/project/ become RAG-ready chunks.
#
# CHUNKING STRATEGY — why it matters:
#   Too large  → retrieval returns a 5-page chapter; signal buried in noise
#   Too small  → retrieval returns half a sentence; no useful context
#   ~1 000 chars with 100-char overlap → retrieves exactly the paragraph
#   the user asked about plus a little context on either side.
#
# METADATA tagging:
#   Every chunk tagged doc_type="truemarkets_official" so we can do
#   source-filtered queries: "search ONLY TrueMarkets docs for price band rules"

TRUE_MARKETS_DOCS = [
    "/mnt/project/About_True_Markets.txt",
    "/mnt/project/Rule_Book.txt",
    "/mnt/project/Price_Bands.txt",
    "/mnt/project/Increment_Sizes.txt",
    "/mnt/project/Exchange_Overview.txt",
    "/mnt/project/Order_Entry.txt",
    "/mnt/project/Market_Data.txt",
    "/mnt/project/Drop_Copy.txt",
    "/mnt/project/TrueX_FIX_API.txt",
    "/mnt/project/TrueX_REST_API.txt",
    "/mnt/project/TrueX_WebSocket_API.txt",
    "/mnt/project/Common_Components.txt",
    "/mnt/project/Administrative.txt",
    "/mnt/project/DeFi_Service_API.txt",
]

CHUNK_SIZE    = 1000
CHUNK_OVERLAP = 100

def ingest_truemarkets_docs():
    out = f"{DIRS['regulatory']}/truemarkets_docs.jsonl"
    all_chunks = []

    for doc_path in TRUE_MARKETS_DOCS:
        if not os.path.exists(doc_path):
            print(f"  ⚠️  Not found (expected in /mnt/project/): {doc_path}")
            continue
        with open(doc_path, encoding="utf-8", errors="replace") as f:
            text = f.read()

        doc_name   = Path(doc_path).stem
        paragraphs = [p.strip() for p in text.split("\n\n") if len(p.strip()) > 50]
        current    = ""
        doc_chunks = 0

        for para in paragraphs:
            if len(current) + len(para) < CHUNK_SIZE:
                current += "\n\n" + para
            else:
                if current.strip():
                    all_chunks.append({"source": doc_name,
                                       "text": current.strip(),
                                       "doc_type": "truemarkets_official"})
                    doc_chunks += 1
                current = current[-CHUNK_OVERLAP:] + "\n\n" + para

        if current.strip():
            all_chunks.append({"source": doc_name,
                               "text": current.strip(),
                               "doc_type": "truemarkets_official"})
            doc_chunks += 1

        print(f"  ✅ {doc_name}: {len(paragraphs)} paragraphs → {doc_chunks} chunks")

    with open(out, "w") as f:
        for chunk in all_chunks:
            f.write(json.dumps(chunk) + "\n")

    print(f"\n  Total: {len(all_chunks)} chunks → {file_size_mb(out)} MB")
    return out

tm_path = ingest_truemarkets_docs()
manifest["sections"]["regulatory"] = {"total_mb": dir_size_mb(DIRS["regulatory"])}
save_manifest()

---
## Section 7 — Crypto News Archives

### Why this data?
News enables sentiment-aware responses and event context:
- *"After the FTX collapse in Nov 2022, BTC dropped 25% in a week"*
- Databricks demo: train a headline sentiment classifier, track in MLflow

CryptoPanic votes (`positive`, `negative`, `important`) give us human-labelled sentiment — no annotation work needed.

In [ ]:
# ─── CELL 7.1 — CryptoPanic News + Sentiment ─────────────────────────────────
# CryptoPanic aggregates 100+ crypto media sources.
# We pull 5 filters × N pages ≈ 20 000 articles with vote-based sentiment labels.
#
# SENTIMENT DERIVATION:
#   votes.positive / votes.negative → bullish / bearish / neutral
#   This becomes training data for a Databricks sentiment classifier:
#     Input:  news headline
#     Output: bullish / bearish / neutral + confidence score
#   Show this in MLflow Experiments during the Databricks demo moment.
#
# Free key at: https://cryptopanic.com/developers/api/
# Without a key, use auth_token="free" for public access.

CRYPTOPANIC_KEY = os.getenv("CRYPTOPANIC_KEY", "free")

def collect_cryptopanic_news(max_pages_per_filter=40):
    out = f"{DIRS['news']}/cryptopanic_news.jsonl"
    existing = 0
    if os.path.exists(out):
        with open(out) as f: existing = sum(1 for _ in f)
        if existing > 5000:
            print(f"  Skipping — already have {existing} news items"); return out

    BASE    = "https://cryptopanic.com/api/v1/posts/"
    filters = ["rising","hot","bullish","bearish","important"]
    articles = []

    for filt in filters:
        params   = {"auth_token": CRYPTOPANIC_KEY, "filter": filt, "public": "true"}
        next_url = BASE
        pages    = 0
        while next_url and pages < max_pages_per_filter:
            try:
                r = requests.get(next_url, params=params, timeout=20)
                if r.status_code == 429:
                    print(f"  Rate limited on {filt}, sleeping 60 s..."); time.sleep(60); continue
                r.raise_for_status()
                data = r.json()
                for item in data.get("results", []):
                    vt   = item.get("votes", {}) or {}
                    pos  = vt.get("positive", 0) or 0
                    neg  = vt.get("negative", 0) or 0
                    sent = "bullish" if pos > neg * 1.5 else ("bearish" if neg > pos * 1.5 else "neutral")
                    articles.append({
                        "id":           item.get("id"),
                        "title":        item.get("title"),
                        "published_at": item.get("published_at"),
                        "source":       (item.get("source") or {}).get("title"),
                        "url":          item.get("url"),
                        "currencies":   [c["code"] for c in item.get("currencies", [])],
                        "filter":       filt,
                        "votes_pos":    pos,
                        "votes_neg":    neg,
                        "votes_imp":    vt.get("important", 0) or 0,
                        "sentiment":    sent,
                    })
                next_url = data.get("next"); params = {}; pages += 1; time.sleep(3)
            except Exception as e:
                log.warning(f"  CryptoPanic ({filt}): {e}"); break
        print(f"  Filter '{filt}': running total {len(articles)}")

    with open(out, "w") as f:
        for a in articles: f.write(json.dumps(a) + "\n")
    print(f"  ✅ {len(articles)} articles saved ({file_size_mb(out)} MB)")
    return out

news_path = collect_cryptopanic_news()
manifest["sections"]["news"] = {"total_mb": dir_size_mb(DIRS["news"])}
save_manifest()

---
## Section 8 — Conversation Training Data (OpenAssistant)

### Why this data?
This is the **most important dataset for the ElevenLabs $7 920 prize**.

ElevenLabs judges on:
1. Natural multi-turn dialogue flow
2. Context retention across turns
3. **Voice-appropriate response length** (1–3 sentences, not a wall of text)

OpenAssistant OASST1 = 161 K human conversations, quality-rated, multi-turn.  
We filter for English + finance topics, reconstruct dialogue threads, and package them for Databricks fine-tuning.

In [ ]:
# ─── CELL 8.1 — OpenAssistant OASST1 Processing ──────────────────────────────
# OASST1 dataset structure:
#   - Stored as a message tree (not pre-formatted conversations)
#   - Each message: {message_id, parent_id, role, text, rank, lang, ...}
#   - rank=0 is the best human-rated response at each position
#
# Our pipeline:
#   1. Filter: English, rank ≤ 1 (top-quality only)
#   2. Reconstruct: walk parent→child chains to get full dialogues
#   3. Finance filter: keyword match on first user message
#   4. Sample: keep ALL finance conversations + 5 000 general
#      (general conversations teach natural turn-taking, greetings, etc.)
#   5. Save: Parquet for Databricks training job
#
# KEY INSIGHT — Voice Length:
#   OASST1 assistant replies avg 200–500 words.
#   For ElevenLabs voice, optimal is 40–80 words.
#   Fine-tune objective: "respond naturally in 2–3 sentences max"
#   This "voice condensing" is a novel angle that impresses ElevenLabs judges.

from datasets import load_dataset

def collect_openassistant():
    out = f"{DIRS['conversation']}/openassistant_processed.parquet"
    if os.path.exists(out):
        print("  Skipping — already collected"); return out

    print("  Loading OASST1 from HuggingFace (~1.5 GB first download)...")
    ds = load_dataset("OpenAssistant/oasst1", split="train")
    df = ds.to_pandas()
    print(f"  Raw: {len(df):,} messages")

    # Step 1: quality filter
    df = df[(df["lang"] == "en") & (df["rank"].fillna(999) <= 1)]
    print(f"  After quality filter: {len(df):,}")

    # Step 2: build conversation trees
    id_to_msg = df.set_index("message_id").to_dict("index")

    def build_chain(mid, depth=0):
        if depth >= 8 or mid not in id_to_msg: return []
        msg = id_to_msg[mid]
        chain = [{"role": msg["role"], "text": msg["text"]}]
        children = df[df["parent_id"] == mid]
        if not children.empty:
            best = children.sort_values("rank").iloc[0]
            chain.extend(build_chain(best["message_id"], depth + 1))
        return chain

    roots = df[df["parent_id"].isna()]["message_id"].tolist()
    print(f"  Building threads from {len(roots):,} roots...")

    convs = []
    for root_id in tqdm(roots[:20000], desc="  Threads"):
        chain = build_chain(root_id)
        if len(chain) >= 2:
            convs.append({
                "thread_id":     root_id,
                "messages":      json.dumps(chain),
                "num_turns":     len(chain) // 2,
                "first_msg":     chain[0]["text"][:300],
            })

    conv_df = pd.DataFrame(convs)

    # Step 3: finance keyword filter
    KEYWORDS = ("bitcoin|crypto|ethereum|blockchain|defi|trading|invest|stock|"
                "market|price|portfolio|finance|money|bank|asset|token|nft|"
                "wallet|exchange|yield|return|risk|hedge|derivative")
    mask    = conv_df["first_msg"].str.lower().str.contains(KEYWORDS, na=False)
    fin_df  = conv_df[mask]
    gen_df  = conv_df[~mask].sample(min(5000, (~mask).sum()), random_state=42)
    final   = pd.concat([fin_df, gen_df]).reset_index(drop=True)
    final.to_parquet(out, index=False, compression="snappy")

    print(f"  Finance threads: {len(fin_df)}")
    print(f"  General threads: {len(gen_df)}")
    print(f"  ✅ Total: {len(final)} conversations saved ({file_size_mb(out)} MB)")
    return out

conv_path = collect_openassistant()
manifest["sections"]["conversation"] = {"total_mb": dir_size_mb(DIRS["conversation"])}
save_manifest()

---
## Section 9 — NFT & Web3 Data

### Why this data?
Completes encyclopedic coverage. Without it the agent can't handle:
*"What's the floor price on Bored Apes?"* or *"Explain ERC-721 vs ERC-1155"*

EIP standard documents also give technical depth for smart contract questions.

In [ ]:
# ─── CELL 9.1 — OpenSea Collections + EIP Token Standards ────────────────────
# OpenSea data (top 500 collections by volume):
#   name, category, total_supply, num_owners,
#   floor_price (ETH), total_volume (ETH), market_cap (ETH)
#
# EIP Standards — critical ones for the voice agent:
#   ERC-20   Fungible token (every altcoin uses this)
#   ERC-721  Non-fungible token — defines unique ownership
#   ERC-1155 Multi-token — both fungible and non-fungible in one contract
#   ERC-2981 NFT royalty standard — how creators earn on secondary sales
#   ERC-4626 Tokenized vaults — backbone of DeFi yield aggregators
#   ERC-4337 Account abstraction — enables smart contract wallets
#
# We pull EIP Markdown directly from ethereum/EIPs GitHub repo.

def collect_opensea(max_collections=500):
    out = f"{DIRS['nft']}/opensea_collections.jsonl"
    if os.path.exists(out):
        print("  Skipping OpenSea — already collected"); return out

    cols = []
    cursor = None
    while len(cols) < max_collections:
        params = {"limit": 100, "order_by": "total_volume"}
        if cursor: params["next"] = cursor
        try:
            r = requests.get("https://api.opensea.io/api/v2/collections",
                             params=params,
                             headers={"accept": "application/json"},
                             timeout=20)
            if r.status_code == 429:
                print("  Rate limited, sleeping 30 s..."); time.sleep(30); continue
            r.raise_for_status()
            data = r.json()
            for c in data.get("collections", []):
                cols.append({
                    "collection": c.get("collection"),
                    "name":       c.get("name"),
                    "description":str(c.get("description", ""))[:500],
                    "category":   c.get("category"),
                    "total_supply":c.get("total_supply"),
                    "num_owners": c.get("num_owners"),
                    "floor_price":(c.get("stats") or {}).get("floor_price"),
                    "total_volume":(c.get("stats") or {}).get("total_volume"),
                    "market_cap": (c.get("stats") or {}).get("market_cap"),
                })
            cursor = data.get("next")
            if not cursor: break
            time.sleep(2)
        except Exception as e:
            log.warning(f"  OpenSea: {e}"); break

    with open(out, "w") as f:
        for c in cols: f.write(json.dumps(c) + "\n")
    print(f"  ✅ {len(cols)} NFT collections saved")
    return out

def collect_eip_standards():
    out = f"{DIRS['nft']}/eip_standards.jsonl"
    if os.path.exists(out):
        print("  EIP standards already collected"); return out

    EIPS = [20, 721, 1155, 2981, 4626, 4337, 4907, 1, 712, 1193, 1271, 2612, 3525]
    records = []
    for num in EIPS:
        url = f"https://raw.githubusercontent.com/ethereum/EIPs/master/EIPS/eip-{num}.md"
        try:
            r = requests.get(url, timeout=15)
            if r.status_code == 200:
                records.append({"eip": num, "url": url, "content": r.text[:5000]})
                print(f"  ✅ EIP-{num}")
        except: pass
        time.sleep(1)

    with open(out, "w") as f:
        for rec in records: f.write(json.dumps(rec) + "\n")
    print(f"  ✅ {len(records)} EIP standards saved")
    return out

collect_opensea()
collect_eip_standards()
manifest["sections"]["nft"] = {"total_mb": dir_size_mb(DIRS["nft"])}
save_manifest()

---
## Section 10 — TrueMarkets Live API Client

### Why this section?
This is **demo-critical**. VoiceFlow needs live market data during the presentation.

We build and test the HMAC-authenticated REST client + WebSocket subscriber here so there are zero surprises on demo day.

In [ ]:
# ─── CELL 10.1 — TrueX REST Client with HMAC Auth ───────────────────────────
# From TrueX REST API docs:
#   Base URL: https://prod.truex.co/api/v1/
#
# HMAC signature payload: timestamp + METHOD + path + body
# All concatenated as strings before encoding.
#
# Example:
#   timestamp = "1706000000"
#   payload   = "1706000000GET/api/v1/instruments"
#   sig       = base64(HMAC-SHA256(api_secret, payload))
#
# Headers required:
#   x-truex-auth-signature  : base64 HMAC sig
#   x-truex-auth-token      : API key ID
#   x-truex-auth-timestamp  : seconds since epoch
#   x-truex-auth-userid     : client identifier
#   X-Truex-Version         : v2026_01_23 (new envelope format)

import hmac, hashlib, base64

TRUEX_BASE   = "https://prod.truex.co/api/v1"
TRUEX_KEY    = os.getenv("TRUEX_API_KEY",    "")
TRUEX_SECRET = os.getenv("TRUEX_API_SECRET", "")
TRUEX_UID    = os.getenv("TRUEX_USER_ID",    "")

def truex_headers(method, path, body=""):
    ts      = str(int(time.time()))
    payload = (ts + method.upper() + path + (body or "")).encode("utf-8")
    sig     = base64.b64encode(
        hmac.new(TRUEX_SECRET.encode("utf-8"), payload, hashlib.sha256).digest()
    ).decode("utf-8")
    return {
        "x-truex-auth-signature": sig,
        "x-truex-auth-token":     TRUEX_KEY,
        "x-truex-auth-timestamp": ts,
        "x-truex-auth-userid":    TRUEX_UID,
        "Content-Type":           "application/json",
        "X-Truex-Version":        "v2026_01_23",
    }

def truex_get(path, auth=True):
    headers = truex_headers("GET", path) if (auth and TRUEX_KEY) else {}
    try:
        r = requests.get(f"{TRUEX_BASE}{path}", headers=headers, timeout=10)
        return r.status_code, r.json() if r.content else {}
    except Exception as e:
        return 0, {"error": str(e)}

# ── Test connectivity ─────────────────────────────────────────────────────────
print("Testing TrueMarkets API connectivity...")
for path in ["/api/v1/health", "/api/v1/instruments", "/api/v1/market/quote"]:
    status, data = truex_get(path, auth=False)
    label = "✅" if status == 200 else f"⚠️  HTTP {status}"
    print(f"  {label}  {path}")
    if status == 200:
        fixture = f"{DIRS['truemarkets']}/{path.split('/')[-1]}_fixture.json"
        with open(fixture, "w") as f: json.dump(data, f, indent=2)
        print(f"       Fixture saved: {fixture}")

manifest["sections"]["truemarkets"] = {"total_mb": dir_size_mb(DIRS["truemarkets"])}
save_manifest()

In [ ]:
# ─── CELL 10.2 — TrueX WebSocket Subscriber ──────────────────────────────────
# TrueX WebSocket channels (from docs):
#   subscribe  → send this to start receiving data
#   instrument → listings, status updates
#   trade      → real-time executed trades
#   ebbo       → Exchange Best Bid/Offer (top-of-book)
#   depth      → full order book depth
#
# UNAUTHENTICATED channels (no API key needed for market data):
#   instrument, trade, ebbo, depth
#
# This cell captures a 10-second snapshot and saves it as a test fixture.
# During the demo you would keep this connection open and feed events
# into the VoiceFlow real-time dashboard.

import asyncio, nest_asyncio
nest_asyncio.apply()   # Allows asyncio.run() inside Jupyter

TRUEX_WS_UAT  = "wss://uat.truex.co/api/v1"
TRUEX_WS_PROD = "wss://prod.truex.co/api/v1"

async def capture_ws(url=TRUEX_WS_UAT, seconds=10):
    try:
        import websockets
    except ImportError:
        print("  Install websockets: pip install websockets"); return []

    sub_msg = json.dumps({
        "type":      "subscribe",
        "channels":  ["instrument","ebbo","trade"],
        "timestamp": int(time.time()),
    })
    messages = []
    try:
        async with websockets.connect(url, open_timeout=10) as ws:
            await ws.send(sub_msg)
            print(f"  Connected to {url} — capturing {seconds} s...")
            deadline = time.time() + seconds
            while time.time() < deadline:
                try:
                    raw = await asyncio.wait_for(ws.recv(), timeout=2.0)
                    msg = json.loads(raw)
                    msg["_captured_at"] = time.time()
                    messages.append(msg)
                except asyncio.TimeoutError:
                    continue
    except Exception as e:
        print(f"  ⚠️  WebSocket unavailable: {e}")
        print("     (Expected if running before hackathon credentials are set up)")

    out = f"{DIRS['truemarkets']}/ws_snapshot.jsonl"
    with open(out, "w") as f:
        for m in messages: f.write(json.dumps(m) + "\n")
    print(f"  ✅ Captured {len(messages)} WebSocket messages")
    return messages

asyncio.run(capture_ws())

---
## Section 11 — Build ChromaDB RAG Index

### Architecture
```
User voice → ElevenLabs STT → query text
  → ChromaDB semantic search → top 5 chunks
  → Claude orchestrator (+ Nemotron for math)
  → response text → ElevenLabs TTS → voice output
```

### Why ChromaDB?
- ✅ Fully local — works offline during the hackathon
- ✅ Sub-100 ms query time — invisible latency in demo
- ✅ Persistent on disk — survives notebook restarts; copy to USB
- ✅ Source-filtered search — restrict queries to `truemarkets_official` docs

In [ ]:
# ─── CELL 11.1 — Build ChromaDB Index ───────────────────────────────────────
# Embedding model: all-MiniLM-L6-v2
#   - 384 dimensions, ~80 MB on disk
#   - 5× faster inference than OpenAI text-embedding-ada-002
#   - Runs fully offline — no API calls during demo
#   - Good performance on both technical documentation and conversational text
#
# INDEXING PRIORITY ORDER:
#   1. TrueMarkets docs  → highest priority; directly tied to sponsor prize
#   2. Wikipedia         → broad accessible knowledge for voice responses
#   3. ArXiv papers      → technical depth for judge probing
#   4. CoinGecko         → coin-specific factual lookups
#   5. DeFi protocols    → ecosystem coverage

import chromadb
from sentence_transformers import SentenceTransformer

EMBED_MODEL = "sentence-transformers/all-MiniLM-L6-v2"
print("Loading embedding model (downloads ~80 MB first time)...")
embedder = SentenceTransformer(EMBED_MODEL)

client = chromadb.PersistentClient(path=DIRS["rag_index"])
try:    client.delete_collection("voiceflow_kb")
except: pass
collection = client.create_collection(
    "voiceflow_kb",
    metadata={"description": "VoiceFlow Intelligence knowledge base",
              "built_at": datetime.now().isoformat()}
)
print("✅ Collection created: voiceflow_kb")

def add_docs(texts, metadatas, ids, batch=100):
    for i in range(0, len(texts), batch):
        t = texts[i:i+batch]; m = metadatas[i:i+batch]; d = ids[i:i+batch]
        embs = embedder.encode(t, show_progress_bar=False).tolist()
        collection.add(documents=t, embeddings=embs, metadatas=m, ids=d)

# ── TrueMarkets docs ─────────────────────────────────────────────────────────
tm_file = f"{DIRS['regulatory']}/truemarkets_docs.jsonl"
if os.path.exists(tm_file):
    texts, metas, ids = [], [], []
    with open(tm_file) as f:
        for i, line in enumerate(f):
            doc = json.loads(line)
            texts.append(doc["text"])
            metas.append({"source_type": "truemarkets", "source": doc["source"]})
            ids.append(f"tm_{i}")
    add_docs(texts, metas, ids)
    print(f"  ✅ Indexed {len(texts)} TrueMarkets chunks")

# ── Wikipedia ────────────────────────────────────────────────────────────────
wiki_file = f"{DIRS['protocols']}/wikipedia_articles.jsonl"
if os.path.exists(wiki_file):
    texts, metas, ids = [], [], []
    with open(wiki_file) as f:
        for i, line in enumerate(f):
            doc = json.loads(line)
            full = doc.get("full_text", "")
            for j in range(0, min(len(full), 30000), 900):
                chunk = full[j:j+1000]
                if len(chunk) > 100:
                    texts.append(chunk)
                    metas.append({"source_type": "wikipedia", "title": doc["title"]})
                    ids.append(f"wiki_{i}_{j}")
    add_docs(texts, metas, ids)
    print(f"  ✅ Indexed {len(texts)} Wikipedia chunks")

# ── ArXiv ────────────────────────────────────────────────────────────────────
arxiv_file = f"{DIRS['protocols']}/arxiv_papers.jsonl"
if os.path.exists(arxiv_file):
    texts, metas, ids = [], [], []
    with open(arxiv_file) as f:
        for i, line in enumerate(f):
            doc = json.loads(line)
            texts.append(f"{doc['title']}\n\n{doc['abstract']}")
            metas.append({"source_type": "academic", "arxiv_id": doc["id"]})
            ids.append(f"arxiv_{i}")
    add_docs(texts, metas, ids)
    print(f"  ✅ Indexed {len(texts)} ArXiv papers")

# ── CoinGecko profiles ───────────────────────────────────────────────────────
cg_file = f"{DIRS['coingecko']}/coin_profiles.jsonl"
if os.path.exists(cg_file):
    texts, metas, ids = [], [], []
    with open(cg_file) as f:
        for i, line in enumerate(f):
            doc = json.loads(line)
            if not doc.get("description"): continue
            cap = doc.get("market_cap_usd")
            texts.append(
                f"{doc['name']} ({str(doc.get('symbol','')).upper()})\n"
                f"Categories: {', '.join(doc.get('categories', []))}\n"
                f"Market Cap: ${cap:,.0f}\n"  if isinstance(cap, (int,float)) else
                f"Market Cap: N/A\n"
                f"Algorithm: {doc.get('hashing_algorithm','N/A')}\n"
                f"{doc['description'][:800]}"
            )
            metas.append({"source_type": "coingecko", "coin_id": doc["id"]})
            ids.append(f"coin_{i}")
    add_docs(texts, metas, ids)
    print(f"  ✅ Indexed {len(texts)} CoinGecko profiles")

# ── DeFi protocols ───────────────────────────────────────────────────────────
defi_file = f"{DIRS['defi']}/all_protocols.parquet"
if os.path.exists(defi_file):
    df_defi = pd.read_parquet(defi_file)
    df_defi = df_defi[df_defi["description"].str.len().fillna(0) > 50].head(1000)
    texts, metas, ids = [], [], []
    for i, row in df_defi.iterrows():
        tvl = row.get("tvl", 0)
        texts.append(
            f"{row['name']} — {row.get('category','N/A')}\n"
            f"TVL: ${tvl:,.0f}\nChains: {row.get('chain','N/A')}\n"
            f"{row['description']}"
        )
        metas.append({"source_type": "defi", "protocol": row["name"]})
        ids.append(f"defi_{i}")
    add_docs(texts, metas, ids)
    print(f"  ✅ Indexed {len(texts)} DeFi protocols")

print(f"\n✅ RAG index complete — {collection.count():,} total documents")
manifest["sections"]["rag_index"] = {"documents": collection.count(),
                                      "total_mb": dir_size_mb(DIRS["rag_index"])}
save_manifest()

In [ ]:
# ─── CELL 11.2 — RAG Query Function (Copy into VoiceFlow Demo) ───────────────
# This is the function you import in your main app.
# Claude agent calls it as a tool to retrieve grounded context.

def rag_query(query_text, n_results=5, source_filter=None):
    """
    Semantic search over the VoiceFlow knowledge base.

    Args:
        query_text    : Natural language question from voice input
        n_results     : How many chunks to retrieve (5 is optimal)
        source_filter : Restrict to one source type:
                        'truemarkets' | 'defi' | 'coingecko'
                        'academic' | 'wikipedia'
    Returns:
        List of {text, source_type, source, distance} dicts
        distance: 0 = perfect match, 1 = unrelated

    Example Claude agent tool call:
        context = rag_query("How do TrueX price bands work?",
                            source_filter="truemarkets")
        # Returns chunks from Rule_Book.txt and Price_Bands.txt
    """
    where  = {"source_type": source_filter} if source_filter else None
    emb    = embedder.encode([query_text]).tolist()
    res    = collection.query(query_embeddings=emb, n_results=n_results,
                              where=where,
                              include=["documents","metadatas","distances"])
    return [
        {"text":        doc,
         "source_type": meta.get("source_type"),
         "source":      meta.get("source") or meta.get("title") or meta.get("coin_id"),
         "distance":    round(dist, 4)}
        for doc, meta, dist in zip(
            res["documents"][0], res["metadatas"][0], res["distances"][0]
        )
    ]

# ── Smoke tests ───────────────────────────────────────────────────────────────
TEST_QUERIES = [
    ("How do TrueX price bands work?",               "truemarkets"),
    ("What is the minimum order notional value?",    "truemarkets"),
    ("Explain Uniswap AMM constant product formula", "defi"),
    ("What is Bitcoin's proof of work?",             "wikipedia"),
    ("Solana market cap and consensus mechanism",    "coingecko"),
]

print("RAG smoke tests\n" + "─"*60)
for q, filt in TEST_QUERIES:
    results = rag_query(q, n_results=2, source_filter=filt)
    print(f"Q: {q}")
    for r in results:
        print(f"  [{r['source_type']}:{r['source']}] d={r['distance']} "
              f"→ {r['text'][:90]}...")
    print()

---
## Section 12 — Validation Report & Cheat Sheet

In [ ]:
# ─── CELL 12.1 — Collection Summary Report ───────────────────────────────────
print("=" * 65)
print("  VOICEFLOW INTELLIGENCE — DATA COLLECTION REPORT")
print(f"  Generated: {datetime.now().strftime('%Y-%m-%d %H:%M UTC')}")
print("=" * 65)

rows = [
    ("01 Market Prices",    DIRS["prices"]),
    ("02 CoinGecko",        DIRS["coingecko"]),
    ("03 DeFi Ecosystem",   DIRS["defi"]),
    ("04 Protocol Docs",    DIRS["protocols"]),
    ("05 Regulatory",       DIRS["regulatory"]),
    ("06 News",             DIRS["news"]),
    ("07 Conversations",    DIRS["conversation"]),
    ("08 NFT & Web3",       DIRS["nft"]),
    ("09 TrueMarkets Live", DIRS["truemarkets"]),
    ("10 RAG Index",        DIRS["rag_index"]),
]

total_mb = 0
print(f"\n  {'Section':<25} {'MB':>8}   Files")
print("  " + "─" * 42)
for name, path in rows:
    mb = dir_size_mb(path)
    total_mb += mb
    n  = len(os.listdir(path)) if os.path.exists(path) else 0
    print(f"  {name:<25} {mb:>8.1f}   {n}")
print("  " + "─" * 42)
print(f"  {'TOTAL':<25} {total_mb:>8.1f} MB")

try:    print(f"\n  RAG index: {collection.count():,} documents")
except: pass

print("\n  PRIZE READINESS")
print("  " + "─" * 42)
checks = [
    ("ElevenLabs  $7 920", os.path.exists(f"{DIRS['conversation']}/openassistant_processed.parquet")),
    ("Databricks  $3 000", os.path.exists(f"{DIRS['prices']}/BTCUSDT_features.parquet")),
    ("TrueMarkets   $500", os.path.exists(f"{DIRS['regulatory']}/truemarkets_docs.jsonl")),
    ("Claude/Nem  $1 000", os.path.exists(DIRS["rag_index"])),
    ("FinTech Trk   $500", os.path.exists(f"{DIRS['defi']}/all_protocols.parquet")),
    ("Best Overall $2 000", total_mb > 1000),
]
for prize, ok in checks:
    print(f"  {'✅' if ok else '❌'}  {prize}")

manifest["completed_at"] = datetime.now().isoformat()
manifest["total_mb"]     = total_mb
save_manifest()
print(f"\n  Manifest → {MANIFEST_PATH}")

In [ ]:
# ─── CELL 12.2 — Hackathon Cheat Sheet ───────────────────────────────────────
SHEET = f"""
╔══════════════════════════════════════════════════════════════╗
║        VOICEFLOW INTELLIGENCE — HACKATHON CHEAT SHEET        ║
╠══════════════════════════════════════════════════════════════╣
║  BASE DIR:  {BASE_DIR}
╠══════════════════════════════════════════════════════════════╣
║  KEY FILE PATHS:                                             ║
║  Prices:        {DIRS['prices']}
║  CoinGecko:     {DIRS['coingecko']}
║  DeFi:          {DIRS['defi']}
║  RAG Index:     {DIRS['rag_index']}
╠══════════════════════════════════════════════════════════════╣
║  TRUEX REST API:   https://prod.truex.co/api/v1/             ║
║  TRUEX WS PROD:    wss://prod.truex.co/api/v1                ║
║  TRUEX WS UAT:     wss://uat.truex.co/api/v1                 ║
║  Auth:  HMAC-SHA256(timestamp + METHOD + path + body)        ║
╠══════════════════════════════════════════════════════════════╣
║  KEY ENDPOINTS:                                              ║
║  GET  /api/v1/health           → service health              ║
║  GET  /api/v1/instruments      → available trading pairs     ║
║  GET  /api/v1/market/quote     → live EBBO                   ║
║  POST /api/v1/orders           → create order                ║
║  GET  /api/v1/orders/active    → open orders                 ║
║  GET  /api/v1/balances         → account balances            ║
╠══════════════════════════════════════════════════════════════╣
║  RAG USAGE:                                                  ║
║  results = rag_query(                                        ║
║      "How do price bands work?",                             ║
║      source_filter="truemarkets"                             ║
║  )                                                           ║
║  context = "\n".join(r["text"] for r in results)             ║
║  # Inject context into Claude agent system prompt            ║
╠══════════════════════════════════════════════════════════════╣
║  PRIZES:  ElevenLabs $7 920 | Databricks $3 000              ║
║           Claude $1 000     | TrueMarkets $500               ║
╚══════════════════════════════════════════════════════════════╝
"""
print(SHEET)
with open(f"{BASE_DIR}/CHEAT_SHEET.txt", "w") as f:
    f.write(SHEET)
print(f"Saved to: {BASE_DIR}/CHEAT_SHEET.txt")